In [ ]:
# Jax imports
import jax
import jax.numpy as jnp
import numpy as np
from jax import jit
from jax import config
config.update("jax_enable_x64", False)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# IO utils
from jax_2dtm.io import load_grid_as_cloud

In [ ]:
# Image simulator
from jax_2dtm.simulator import rotate_and_translate, project, anti_aliasing_filter
from jax_2dtm.simulator import Cloud, Pose, ImageConfig
from jax_2dtm.simulator import ScatteringModel
from jax_2dtm.simulator import ParameterState
from jax_2dtm.utils import fft, ifft, fftfreqs

In [ ]:
# Volume filename and metadata
pixel_size = 5.28 # Angstroms
filename = "../tests/data/3jar_13pf_bfm1_ps5_28.mrc"

In [ ]:
# Configure image data and read template as point cloud
config = ImageConfig((60, 60), pixel_size, eps=1e-4)
cloud = load_grid_as_cloud(filename, config, threshold=1e-4)

In [ ]:
# Compute scattering image
freqs = fftfreqs(config.shape, config.pixel_size)
pose = Pose(np.pi/8, np.pi/10, np.pi/4, 50.0, 50.0)
transformed_cloud = rotate_and_translate(cloud, pose)
scattering_image = anti_aliasing_filter(freqs, config.pixel_size) * project(transformed_cloud, config)

In [ ]:
# Plot scattering
plt.imshow(ifft(scattering_image), origin="lower", cmap="gray")

In [ ]:
# Sanity check that ffts are correct
plt.imshow(ifft(fft(ifft(scattering_image))) - ifft(scattering_image), origin="lower", cmap="gray")
plt.colorbar()

Now that we have confirmed the pipeline works step-by-step, let's demonstrate how to use the API.

In [ ]:
# Initialize model, parameters, and compute image
state = ParameterState(pose)
model = ScatteringModel(config, cloud, state)
scattering_image = model(state)
plt.imshow(ifft(scattering_image), origin="lower", cmap="gray")

Computing an image is straight-forward, but really we want to define a function that can be arbitrarily transformed by JAX and evaulated at subsets of the parameters.

In [ ]:
# Define subset of parameters over which to evaluate model
pose = Pose(np.pi/8, np.pi/12, np.pi/4, 50.0, 50.0)
state = ParameterState(pose=pose)
params = dict(view_phi=np.pi)

In [ ]:
# Define jitted pipeline, first defining the model globally
model = ScatteringModel(config, cloud, state)

@jax.jit
def compute_image(params):
    scattering_image = model(params)
    return scattering_image

In [ ]:
# Benchmark jitted pipeline
%timeit scattering_image = compute_image(params).block_until_ready()

In [ ]:
# Benchmark non-jitted pipeline
%timeit scattering_image = model(params)